In [72]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt
import nibabel as nib

from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# Some constants 
root_folder = "/docs/src/kt/data"
series_folder = "/docs/src/kt/data_by_series"
resized_series_folder = "/docs/src/kt/data_by_series_resized"
IMAGES_PATH = "/docs/src/kt/data_by_series_resized_images"
dst_folder = "/docs/src/kt/nii_resized"
MIN_NUMBER_OF_PHOTO = 100
EMPTY_VALUE = 0

DEBUG = 1

patients = os.listdir(series_folder)
patients.sort()

curr_patient = "Dimakova A.I. - Body 1.0"
curr_scan_set = "16"


In [132]:
def load_scan(path):
    slices = [pydicom.dcmread(path + "/" + _) for _ in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness  = slice_thickness

    return slices

def get_pixel_hu(slices):
    image = np.stack([_.pixel_array for _ in slices])
    image = image.astype(np.int16)

    image[image == -2000] = 0

    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope

        if slope != 1:
            image[slice_number] = slope * image[slice_number].as_type(np.float64)
            image[slice_number] = image[slice_number].as_type(np.int16)

        image[slice_number] += np.int16(intercept)

    return np.array(image, dtype=np.int16)
    
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)
    if(DEBUG == 1):
        print("curr spacing:", spacing)

    resize_factor = spacing / new_spacing
    new_shape = np.round(image.shape * resize_factor)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor

    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')

    return image, new_spacing

def segment_to_range(image, low_range, high_range):
    binary_image = np.array((image >= low_range) & (image <= high_range), dtype=np.int8)
    return binary_image

def save_to_nifti(fname, image, spacing):
    nib_img = nib.Nifti1Image(image, spacing)
    nib.save(nib_img, dst_folder + "/" + fname + ".nii")

In [139]:
def load_and_segment_patient(folder, curr_patient, curr_scan_set, new_spacing = [1,1,1]):
    slices = load_scan(folder + "/" + curr_patient + "/" + curr_scan_set)
    print("slices shape:", np.array(slices).shape)
    image = get_pixel_hu(slices)
    if(DEBUG == 1):
        print("original image shape", image.shape)

    resampled_image, new_spacing = resample(image, slices, new_spacing)
    if(DEBUG == 1):
        print("new spacing", new_spacing)
        print("resampled image shape", resampled_image.shape)
    
    #     nib_img = nib.Nifti1Image()
    #     nib.save(nib_img, dst_folder + "/" + curr_patient + "__" + curr_scan_set + ".nii")

    slices_2 = np.stack([_ for _ in resampled_image], axis=2)
    save_to_nifti(curr_patient + "__" + curr_scan_set, slices_2, np.eye(4) * np.append(new_spacing, [1]))
    
    bin_mask = segment_to_range(slices_2, 20, 50)
    slices_2[bin_mask == 0] = EMPTY_VALUE
    save_to_nifti("mask/" + curr_patient + "__" + curr_scan_set, slices_2, np.eye(4) * np.append(new_spacing, [1]))
    
    
    return resampled_image

In [140]:
resampled_image = load_and_segment_patient(series_folder, curr_patient, curr_scan_set, [2,2,2])

slices shape: (532,)
original image shape (532, 512, 512)
curr spacing: [0.8   0.782 0.782]
new spacing [1.9981221  2.00192001 2.00192001]
resampled image shape (213, 200, 200)


In [131]:
im1 = np.array(np.eye(5) * 8, dtype=np.int8)
bim1 = segment_to_range(im1, 4, 10)
bim1[0, 0] = bim1[4,4] = 0
im1[bim1 == 0] = 0
im1

array([[0, 0, 0, 0, 0],
       [0, 8, 0, 0, 0],
       [0, 0, 8, 0, 0],
       [0, 0, 0, 8, 0],
       [0, 0, 0, 0, 0]], dtype=int8)